In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

df = pd.read_csv('dataset/ProcYelp.csv', index_col=0, low_memory=False)
df.head(5)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,hours.Wednesday,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,12:0-20:0,...,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,11:0-2:0,...,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,10:30-21:0,...,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,9:0-9:15,...,0,0,0,0,0,0,3,3,0,0
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57,11:0-21:0,...,0,0,0,0,0,1,0,0,0,0


## Zadání a očekávané otázky

### Zadání:
    Objednala si vás národní turistická agentura, abyste zhodnotili vybraná města a jejich návštěvníky podle recenzí podniků.
### Otázky
     Efa - Rozdělit recenze na pozitivní/negativní za použití SpaCY a pak dál řešit ty atributy města atd.?
        
        Jak se liší ->Podniky
                    ->Kategorie podniků dle města
                    ->Kategorie podniků a jak/jestli se liší jejich hodnocení
                    ->Podniky a jak/jestli se liší jejich hodnocení na základě různých vlastností (attributes, meals, ambience)
                    ->
                    ->Zákazníci
                    ->recenz zákazníků dle města (keywords, stars)
                    -> atd. nevím, seru na to dneska

In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mates\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
from nltk import tokenize
nltk.download('punkt')
def getScore(text):
    results = {}
    tokenized = tokenize.sent_tokenize(text)
    for sentence in tokenized:
        result_dict = sid.polarity_scores(sentence)
        result_dict.pop('compound', None)
        result_dict.pop('neu', None)
        max_key = max(result_dict, key=result_dict.get)
        if max_key in results:
            results[max_key] += 1
        else:
            results[max_key] = 1

    return max(results, key=results.get)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mates\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df['score'] = df.apply(lambda x: getScore(x.text), axis = 1)

In [5]:
df["score"].value_counts()

pos    139465
neg     75409
Name: score, dtype: int64

In [6]:
df.groupby("city")['score'].value_counts()

city      score
Atlanta   pos      24063
          neg      13832
Austin    pos      46177
          neg      22909
Boston    pos      27976
          neg      16092
Orlando   pos      16688
          neg      10426
Portland  pos      24561
          neg      12150
Name: score, dtype: int64

In [7]:
# df['score'] = df['score'].map({'pos': True, 'neg': False})

In [8]:
with open("test.txt", 'r') as f:
    x = f.read()
import ast
x = ast.literal_eval(x)

In [9]:
# def checkIfExists(categories, item):
#     if f"{item}," in categories:
#         return 1
#     return 0

In [10]:
# for item in x:
#     df[f"Category_{item}"] = df.apply(lambda x: checkIfExists(x.categories, item), axis = 1)

In [11]:
df["Category_Restaurants"].value_counts()

1    157065
0     57809
Name: Category_Restaurants, dtype: int64

In [12]:
values = []
for item in x:
    val = df[f"Category_{item}"].value_counts().to_dict()
    val['Category'] = f"Category_{item}"
    new_dict = {}
    for key, value in val.items():
        if key != 0:
            new_dict[str(key)] = value
    if "1" in new_dict:
        values.append(new_dict)
    # print(f"Category_{item} values: \n{val}")
values

[{'1': 6391, 'Category': 'Category_Active Life'},
 {'1': 2576, 'Category': 'Category_Afghan'},
 {'1': 783, 'Category': 'Category_African'},
 {'1': 501, 'Category': 'Category_Airlines'},
 {'1': 934, 'Category': 'Category_Airport Shuttles'},
 {'1': 1180, 'Category': 'Category_Airports'},
 {'1': 45661, 'Category': 'Category_American (New)'},
 {'1': 34802, 'Category': 'Category_American (Traditional)'},
 {'1': 1522, 'Category': 'Category_Amusement Parks'},
 {'1': 368, 'Category': 'Category_Appliances & Repair'},
 {'1': 409, 'Category': 'Category_Architectural Tours'},
 {'1': 1344, 'Category': 'Category_Argentine'},
 {'1': 553, 'Category': 'Category_Art Galleries'},
 {'1': 553, 'Category': 'Category_Art Museums'},
 {'1': 11366, 'Category': 'Category_Arts & Entertainment'},
 {'1': 11401, 'Category': 'Category_Asian Fusion'},
 {'1': 401, 'Category': 'Category_Auto Detailing'},
 {'1': 1158, 'Category': 'Category_Auto Repair'},
 {'1': 557, 'Category': 'Category_Auto Upholstery'},
 {'1': 2353, '

In [13]:
from heapq import nlargest
nlargest(10, values, key=lambda item: item['1'])

[{'1': 157065, 'Category': 'Category_Restaurants'},
 {'1': 79392, 'Category': 'Category_Bars'},
 {'1': 66468, 'Category': 'Category_Nightlife'},
 {'1': 60351, 'Category': 'Category_Food'},
 {'1': 45661, 'Category': 'Category_American (New)'},
 {'1': 34802, 'Category': 'Category_American (Traditional)'},
 {'1': 29513, 'Category': 'Category_Breakfast & Brunch'},
 {'1': 25501, 'Category': 'Category_Sandwiches'},
 {'1': 19679, 'Category': 'Category_Cocktail Bars'},
 {'1': 17955, 'Category': 'Category_Burgers'}]

In [14]:
df.groupby(["city", "Category_Restaurants"])['score'].value_counts()

city      Category_Restaurants  score
Atlanta   0                     pos       6844
                                neg       4514
          1                     pos      17219
                                neg       9318
Austin    0                     pos      10900
                                neg       6047
          1                     pos      35277
                                neg      16862
Boston    0                     pos       7521
                                neg       4391
          1                     pos      20455
                                neg      11701
Orlando   0                     pos       4284
                                neg       4229
          1                     pos      12404
                                neg       6197
Portland  0                     pos       5991
                                neg       3088
          1                     pos      18570
                                neg       9062
Name: score, dtype: in

In [17]:
df['date']

0         2011-10-27 14:32:57
1         2015-10-04 05:37:01
2         2017-05-26 03:05:46
3         2017-09-20 15:38:27
4         2012-11-06 07:09:57
                 ...         
214869    2018-04-17 00:28:02
214870    2017-10-13 21:02:12
214871    2016-10-31 20:47:16
214872    2015-07-21 13:21:59
214873    2014-01-04 03:19:41
Name: date, Length: 214874, dtype: object

In [16]:
df.to_csv('dataset/readyYelp.csv')